In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 载入数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True) # 下载数据

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
# 批次大小
batch_size = 100
number_neural = 500
number_neural_1 = 300
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size # // 整除

# 定义两个 placeholder
x = tf.placeholder(tf.float32, [None, 784]) # None 表示任意维度
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
lr = tf.Variable(0.001, dtype=tf.float32) # tf.assign

# 创建一个简单的神经网络
W_1 = tf.Variable(tf.truncated_normal([784,number_neural], stddev=0.1)) # 截断的正太分布
b_1 = tf.Variable(tf.zeros([number_neural])+0.1) # 初始化偏置值为0.1
L_1 = tf.nn.tanh(tf.matmul(x, W_1) + b_1)
L_1_dropout = tf.nn.dropout(L_1, keep_prob) # keep_prob = 0.8 80% 的神经元工作

W_2 = tf.Variable(tf.truncated_normal([number_neural,number_neural_1], stddev=0.1)) # 截断的正太分布
b_2 = tf.Variable(tf.zeros([number_neural_1])+0.1) # 初始化偏置值为0.1
L_2 = tf.nn.tanh(tf.matmul(L_1_dropout, W_2) + b_2)
L_2_dropout = tf.nn.dropout(L_2, keep_prob) # keep_prob = 0.8 80% 的神经元工作

# 输出层
W_3 = tf.Variable(tf.truncated_normal([number_neural_1,10], stddev=0.1)) # 截断的正太分布
b_3 = tf.Variable(tf.zeros([10])+0.1) # 初始化偏置值为0.1

predict = tf.nn.softmax(tf.matmul(L_2_dropout,W_3)+b_3)

# softmax
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predict))
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
train_step = tf.train.AdamOptimizer(lr).minimize(loss) # 收敛速度会比SGD快
# lr 设置为 1e-2 时 结果较差, 92% 左右震荡, 不能进一步收敛
# lr 设置为 1e-5 时 初始结果较差, 66% , 收敛速度较慢, 但个batch 训练时间长

# 初始化变量
init = tf.global_variables_initializer()

# 结果存放在一个bool 型列表中
# tf.argmax(variable, axis) axis = 0 在列中找最大的数, axis = 1 在行中找最大的数
# tf.equal https://blog.csdn.net/yjk13703623757/article/details/77251042
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(predict,1)) # tf.equal 
# 求准确率
arruracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31):
        
        sess.run(tf.assign(lr, 0.001*(0.95**epoch)))
        
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        
        learning_rate = sess.run(lr)
        
        train_acc = sess.run(arruracy, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:0.8})# 70% 工作
        test_acc = sess.run(arruracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})# 测试的时候所有的神经元都工作
        print("Iter " + str(epoch) + "\t Training Accuracy" + str(train_acc) + "\t Testing Accuracy" + str(test_acc) + "\t Learning Rate" + str(learning_rate))

Iter 0	 Training Accuracy0.94692725	 Testing Accuracy0.9508	 Learning Rate0.001
Iter 1	 Training Accuracy0.9602727	 Testing Accuracy0.962	 Learning Rate0.00095
Iter 2	 Training Accuracy0.9717091	 Testing Accuracy0.9692	 Learning Rate0.0009025
Iter 3	 Training Accuracy0.9760909	 Testing Accuracy0.9713	 Learning Rate0.000857375
Iter 4	 Training Accuracy0.9792182	 Testing Accuracy0.9721	 Learning Rate0.00081450626
Iter 5	 Training Accuracy0.98081815	 Testing Accuracy0.9743	 Learning Rate0.0007737809
Iter 6	 Training Accuracy0.98112726	 Testing Accuracy0.9745	 Learning Rate0.0007350919
Iter 7	 Training Accuracy0.98449093	 Testing Accuracy0.9776	 Learning Rate0.0006983373
Iter 8	 Training Accuracy0.9857818	 Testing Accuracy0.978	 Learning Rate0.0006634204
Iter 9	 Training Accuracy0.9876	 Testing Accuracy0.978	 Learning Rate0.0006302494
Iter 10	 Training Accuracy0.98892725	 Testing Accuracy0.98	 Learning Rate0.0005987369
Iter 11	 Training Accuracy0.98874545	 Testing Accuracy0.9783	 Learning 